# Training the Faster RCNN model with resnet50 backbone

In [ ]:
import os
import torch
import torchvision
import xml.etree.ElementTree as ET
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchmetrics.detection.mean_ap import MeanAveragePrecision


# Config 
CLASSES = ["__background__", "donkey", "tortoise"]
NUM_CLASSES = len(CLASSES)

# Force CPU to avoid CUDA loading issues
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print("Using device:", DEVICE)

#VOC Dataset Loader
class VOCDataset(Dataset):
    def __init__(self, img_dir, ann_dir, transform=None):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.transform = transform
        self.images = [f for f in os.listdir(img_dir) if f.endswith((".jpg", ".png"))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_filename = self.images[idx]
        img_path = os.path.join(self.img_dir, img_filename)
        ann_path = os.path.join(self.ann_dir, os.path.splitext(img_filename)[0] + ".xml")

        image = Image.open(img_path).convert("RGB")
        w, h = image.size

        boxes = []
        labels = []

        tree = ET.parse(ann_path)
        root = tree.getroot()

        for obj in root.findall("object"):
            label = obj.find("name").text
            if label not in CLASSES:
                continue
            labels.append(CLASSES.index(label))

            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)
            boxes.append([xmin, ymin, xmax, ymax])

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
        }

        if self.transform:
            image = self.transform(image)

        return image, target

# Transforms
def get_transform():
    return ToTensor()


root = "C:/Users/anagh/OneDrive/Documents/MAI/S3/ACV/Toy_Data"
train_dataset = VOCDataset(
    img_dir=os.path.join(root, "train", "images"),
    ann_dir=os.path.join(root, "train", "annotations"),
    transform=get_transform()
)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Load Faster R-CNN Model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, NUM_CLASSES)
model.to(DEVICE)

#Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training and Evaluation 
EPOCHS = 10
#print(" Starting training...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images, targets in train_loader:
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f" Epoch {epoch+1}/{EPOCHS} | Total Loss: {total_loss:.4f}")

  
    model.eval()
    metric = MeanAveragePrecision()
    with torch.no_grad():
        for images, targets in train_loader:
            images = [img.to(DEVICE) for img in images]
            outputs = model(images)
            outputs = [{k: v.cpu() for k, v in t.items()} for t in outputs]
            targets = [{k: v.cpu() for k, v in t.items()} for t in targets]
            metric.update(outputs, targets)

        results = metric.compute()
        print(f" mAP@IoU=0.5: {results['map_50']:.4f} | mAP@IoU=0.5:0.95: {results['map']:.4f}")

torch.save(model.state_dict(), "fasterrcnn_pascalvoc.pth")
print("Model saved as fasterrcnn_pascalvoc.pth")


Using device: cuda
 Starting training...
 Epoch 1/10 | Total Loss: 25.0001
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.8527
 Epoch 2/10 | Total Loss: 7.4475
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9062
 Epoch 3/10 | Total Loss: 5.5346
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9301
 Epoch 4/10 | Total Loss: 4.6425
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9483
 Epoch 5/10 | Total Loss: 4.0596
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9111
 Epoch 6/10 | Total Loss: 3.9126
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9343
 Epoch 7/10 | Total Loss: 3.4913
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9614
 Epoch 8/10 | Total Loss: 3.0033
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9719
 Epoch 9/10 | Total Loss: 2.7801
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9689
 Epoch 10/10 | Total Loss: 2.6654
 mAP@IoU=0.5: 0.9950 | mAP@IoU=0.5:0.95: 0.9654
Model saved as fasterrcnn_pascalvoc.pth


# Inferencing

In [ ]:

import cv2
import time
import torch
import torchvision
from torchvision.transforms import functional as F


CLASSES = ["__background__", "donkey", "tortoise"]
NUM_CLASSES = len(CLASSES)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, NUM_CLASSES)
model.load_state_dict(torch.load("fasterrcnn_pascalvoc.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()


input_path = r"C:\Users\anagh\OneDrive\Documents\MAI\S3\ACV\Inference_Video.mp4"
output_path = r"C:\Users\anagh\OneDrive\Documents\MAI\S3\ACV\Inference_outputVideo.mp4"

cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


CONF_THRESHOLD = 0.5
frame_count = 0
total_time = 0

print(" Starting video inference...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # Convert to tensor
    image_tensor = F.to_tensor(frame).to(DEVICE)
    with torch.no_grad():
        prediction = model([image_tensor])[0]

    # Draw predictions
    for box, score, label in zip(prediction['boxes'], prediction['scores'], prediction['labels']):
        if score >= CONF_THRESHOLD:
            x1, y1, x2, y2 = box.int().tolist()
            class_name = CLASSES[label]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name} {score:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    end_time = time.time()
    total_time += (end_time - start_time)
    frame_count += 1

    out.write(frame)
    cv2.imshow("Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Summary 
avg_fps = frame_count / total_time if total_time > 0 else 0
print(f"\n Inference complete. Output saved to {output_path}")
print(f" Processed {frame_count} frames in {total_time:.2f} seconds")
print(f" Average Inference Speed: {avg_fps:.2f} FPS")


Using device: cuda
🔍 Starting video inference...

 Inference complete. Output saved to C:\Users\anagh\OneDrive\Documents\MAI\S3\ACV\Inference_outputVideo.mp4
 Processed 2845 frames in 423.72 seconds
 Average Inference Speed: 6.71 FPS
